In [3]:
from pgmpy.models import BayesianNetwork
def construct_monty_model():
    model: BayesianNetwork = BayesianNetwork()
    model.add_nodes_from(["guest", "car", "host"])
    model.add_edge("guest", "host")
    model.add_edge("car", "host")
    return model

monty = construct_monty_model()

In [6]:
from pgmpy.factors.discrete import TabularCPD
cpd_g = TabularCPD(
    variable="guest", variable_card=3, values=[[0.33333], [0.33333], [0.33333]]
)
cpd_c = TabularCPD(
    variable="car", variable_card=3, values=[[0.33333], [0.33333], [0.33333]]
)
cpd_h = TabularCPD(
    variable="host",
    variable_card=3,
    values=[
            [0, 0, 0, 0, 0.5, 1, 0, 1, 0.5],
            [0.5, 0, 1, 0, 0, 0, 1, 0, 0.5],
            [0.5, 1, 0, 1, 0.5, 0, 0, 0, 0],
    ],
    evidence=["guest", "car"],
    evidence_card=[3, 3],
)

In [7]:
monty = construct_monty_model()
monty.add_cpds(cpd_g, cpd_c, cpd_h)

In [8]:
monty.check_model()

True

In [10]:
import pygraphviz as pgv
def dagviz(nodes, edges, name):
    G = pgv.AGraph(strict=False, directed=True)
    G.add_nodes_from(nodes)
    for edge in edges:
        G.add_edge(edge)
    G.write(name + ".dot")
    # use dot
    G.layout(prog="dot")
    # write previously positioned graph to PNG file
    G.draw(name + ".png")

In [11]:
def viz_model(model, filename):
    dagviz(model.nodes, model.edges, filename)

In [12]:
from pgmpy.inference import BeliefPropagation
viz_model(monty, "monty")

In [14]:
from pgmpy.inference import BeliefPropagation
belief_propagation = BeliefPropagation(monty)
res = belief_propagation.query(variables=["car"], evidence={"guest": 0, "host": 1})
print(res)

+--------+------------+
| car    |   phi(car) |
+========+============+
| car(0) |     0.3333 |
+--------+------------+
| car(1) |     0.0000 |
+--------+------------+
| car(2) |     0.6667 |
+--------+------------+
